In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

# compilation en slides : 
# jupyter nbconvert prez_artefact.ipynb --to slides --reveal-prefix reveal.js-3.1.0

#  Retour d'expérience : Artefact


# Meetup Lille.py (26 avril 2017)


[Github du projet](https://github.com/bidouilleurslibristes/Artefact)

Faire un jeu tangible en python et en mode maker.


* Matthieu Falce
* Yoann Dufresne 
* Camille Lihouck 
* Pierre Marijean
* Jordan Bouchoucha
* et beaucoup d'autres

-------------

Association des Bidouilleurs Libristes (ABL)


## Présentation du projet

* Installation interactive ludique. 
* Enigmes à découvrir et résoudre


In [23]:
HTML("""
<video width="600" controls>
  <source src="./images/artefact_video.webm" type="video/webm">
</video>
""")

In [16]:
Image(url= "./images/artefact-1.jpg", width=300)

In [15]:
Image(url= "./images/artefact-3.jpg", width=800)

In [14]:
Image(url= "./images/artefact-2.jpg", width=800)

# Aspect technique 

_Pouvoir débrancher n'importe quel cable, le rebrancher et le jeu se relance_

## Matériel

 * Arduino * 8 (gestion des entrées / sorties)
 * Raspberry pi * 3 (gestion du jeu et des communications)
 * entrée / sortie (80 boutons + 80 leds + 8 rubans de 32 leds)
 * alimentation par cable internet (PoE) 

## Logiciel


  

 * Codé en python3 (pas d'asyncio)
 * Multiplateforme
 * Installation / déploiement avec Ansible

Ansible
 * ultra simple de faire une nouvelle installation quand on change de raspi
 * problèmes quand pas internet, il faut prévoir ce cas (module `copy`)
 * relation amour / haine avec Ansible

### Architecture

* Réseau maître (gère la logique) <-> esclaves (connectés aux arduino) 
* Communication réseau par ZMQ
* Avahi (linux) pour DNS local
* raspi <-> arduino : communicaion série (pySerial) + protocole custom (noms + reconnections)


![Architecture réseau](./images/Network_archi_artefact.png)


** ZMQ + Matthieu = 💖 ** 
 * reconnexions automatiques
 * PUB / SUB => simplifie le paramétrage / on peut perdre des messages... 
 * ZMQc => envoie / écoute de messages

 * WEB SCALABLE \o/ 
 * ajout simple d'un esclave
 * possibilité d'espionner master -> esclave
 * simulation de matériel => messages customs

  
### Jeu

 * une partie => série d'énigmes composées d'une ou plusieurs sous-enigmes 
 * 1 bouton géré par seulement 1 sous-enigme
 * rubans gérés par plusieurs sous-enigmes
 
 -------------------------------
 
 * logique en python (avec les micro-services ZMQ, pourrait être en autre chose)
 * abstraction du matériel / page web (Flask) => développer sans installation complète 

💔

* typage dynamique (bug à l'éxecution...)
* problème pour faire des enigmes temporelles (résolu par asyncio ?)
* essai de lecture de musiques en python => échec complet (problèmes de son et de grésillements, probablement dus au GIL...)
* difficulté de conceptions des énigmes

### Analyse des parties

 * logs rajouté le 2ème jour
 * surveiller la partie en cours sans rentrer dans la salle
 * déséquilibres dans le jeu
 * "replay" des parties


#### Débug live / haut niveau : 

proTip 🔬 : watch -n 5 "python parse_log.py game_log/7.log"

    Fichier : c:\Users\falce\Downloads\zoomachine\ZooMachine-3\master\game_log/7.log
    unknown event:reboot
    durée totale (hh:mm:ss) : 0:39:44.569086
      * enigme 0 résolue en 0:00:00 (hh:mm:ss) / 0 erreurs
      * enigme 1 résolue en 0:00:46.106088 (hh:mm:ss) / 0 erreurs
      * enigme 2 résolue en 0:00:24.634655 (hh:mm:ss) / 4 erreurs
      * enigme 3 résolue en 0:00:18.149378 (hh:mm:ss) / 1 erreurs
      * enigme 4 résolue en 0:00:10.015720 (hh:mm:ss) / 0 erreurs
      * enigme 5 résolue en 0:00:32.694306 (hh:mm:ss) / 1 erreurs
      * enigme 6 résolue en 0:04:58.726424 (hh:mm:ss) / 18 erreurs
      * enigme 7 résolue en 0:01:34.790013 (hh:mm:ss) / 6 erreurs
      * enigme 8 résolue en 0:05:21.960151 (hh:mm:ss) / 13 erreurs
      * enigme 9 résolue en 0:06:34.323552 (hh:mm:ss) / 7 erreurs
      * enigme 10 résolue en 0:00:09.616676 (hh:mm:ss) / 0 erreurs
      * enigme 11 résolue en 0:00:06.726740 (hh:mm:ss) / 0 erreurs
      * enigme 12 résolue en 0:00:44.977521 (hh:mm:ss) / 3 erreurs
      * enigme 13 résolue en 0:03:53.741781 (hh:mm:ss) / 12 erreurs
      * enigme 14 résolue en 0:01:11.693027 (hh:mm:ss) / 0 erreurs
      * enigme 15 résolue en 0:01:13.598689 (hh:mm:ss) / 2 erreurs
      * enigme 16 résolue en 0:05:32.754788 (hh:mm:ss) / 11 erreurs
      * enigme 17 résolue en 0:01:04.828428 (hh:mm:ss) / 1 erreurs
      * enigme 18 résolue en 0:03:37.717643 (hh:mm:ss) / 1 erreurs


#### Analyse plus fine

Stack utilisée : 
 * pandas
 * matplotlib / seaborn

![Analyse globale](./images/analysis-1.png)

![Pourcentage de continuation](./images/analysis-2.png)

![Analyse erreurs différentes enigmes](./images/analysis-3.png)

# Conclusion

 * le projet à été un succès
  * au niveau technique => ça fonctionne
  * au niveau ludique => ça plait aux joueurs
  * n'a pas planté en production

** On peut débrancher n'importe quel cable, le rebrancher en continuant à marcher. **


* penser au pire (en étant réaliste)
* pouvoir débugguer facilement ses "microservices"
* utiliser / créer des outils et méthodes pour débugguer
* se demander "Comment je peux débugguer ça ?" pour chaque choix (espionnage, simulation, abstraction matériel...)

 * python n'a pas été trop problématique 
  * 👍 code "compréhensible"
  * 👍 multiplateforme "accidentel"
  * 👍 écosystème mature et robuste (ipdb 💖)
  * 👍 flexible (faire tourner une appli flask dans une thread)
  * 😱 dynamicité et des "surprises" à l'exécution
  * 😱 gestion des threads (je ne sais pas les fermer, besoin de 6 ctrl-C pour killer le programme...)
  * 😱 audio => il faudrait regarder plus dans les détails

----------------------------

 * services => systemd 

 * à venir (peut-être)
  * tests unitaires / d'intégration
  * documentation
  * pyrasite pour shell live